In [1]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import numpy as np
import bz2
import csv
import io
import json
import re
import time
import random
import requests
import datetime
from pathlib import Path
from pprint import pprint
from typing import List, Dict
from dateutil.relativedelta import relativedelta
import lsde2021.csv as csvutils
import lsde2021.utils as utils
import lsde2021.download as dl
from pyspark.sql import SparkSession
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [2]:
MAX_MEMORY = "30G"

spark = SparkSession \
    .builder \
    .appName("parse-wikipedia-sql-dumps") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .config('spark.driver.maxResultSize', MAX_MEMORY) \
    .config('spark.ui.showConsoleProgress', 'false') \
    .getOrCreate()
sc = spark.sparkContext

csv_loader = spark.read.format("csv").options(header='True', inferSchema='True')
parquet_reader = spark.read.format("parquet").options(inferSchema='True')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/10/27 12:08:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
pageview_complete_processed_src = Path("../nvme/pageview_complete_processed")
# pageview_complete_per_topic_dest = Path("../nvme/pageview_complete_per_topic")
end_date = datetime.date(2021, 10, 1)

daily_pageview_files = []
for year in [2019]: # 2019, 2020, 2021]:
    daily_range = list(dl.date_range(
        datetime.date(year, 1, 1),
        datetime.date(year, 12, 31),
    ))
    
    daily_range = [d for d in daily_range if (end_date - d).total_seconds() > 0]
    daily_pageview_files += daily_range
    
daily_pageview_files = [
    (
        pageview_complete_processed_src / Path("/".join(dl.wikimedia_pageview_complete_local_file(date, monthly=False))).with_suffix(".parquet"),
        date,
        # pageview_complete_per_topic_dest / Path("/".join(dl.wikimedia_pageview_complete_local_file(date, monthly=False))).with_suffix(".parquet"),
    )
    for date in daily_pageview_files
]
pprint(daily_pageview_files[:10])

[(PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190101-user.parquet'),
  datetime.date(2019, 1, 1)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190102-user.parquet'),
  datetime.date(2019, 1, 2)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190103-user.parquet'),
  datetime.date(2019, 1, 3)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190104-user.parquet'),
  datetime.date(2019, 1, 4)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190105-user.parquet'),
  datetime.date(2019, 1, 5)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190106-user.parquet'),
  datetime.date(2019, 1, 6)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190107-user.parquet'),
  datetime.date(2019, 1, 7)),
 (PosixPath('../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190108-user.parquet'),
  datetime.dat

In [4]:
group_cols = ["topic", "dbname", "wiki_code", "group", "language"]

def prepare(daily_processed_file, date):
    d = date.timetuple()
    day = f"{d.tm_year}-{d.tm_mon}-{d.tm_mday}"
    # print(day)
    
    df = spark.read.format("parquet").load(str(daily_processed_file))
    
    topic1_counts = df.select(*group_cols, "daily_total", F.explode("topics1").alias("topic"))
    topic1_counts = topic1_counts.groupBy(group_cols).agg(F.sum("daily_total").alias(day))
    topic1_counts = topic1_counts.withColumn("level", F.lit(1))
    # topic1_counts.limit(10).show()
    
    topic2_counts = df.select(*group_cols, "daily_total", F.explode("topics2").alias("topic"))
    topic2_counts = topic2_counts.groupBy(group_cols).agg(F.sum("daily_total").alias(day))
    topic2_counts = topic2_counts.withColumn("level", F.lit(2))
    
    topic3_counts = df.select(*group_cols, "daily_total", F.explode("topics3").alias("topic"))
    topic3_counts = topic3_counts.groupBy(group_cols).agg(F.sum("daily_total").alias(day))
    topic3_counts = topic3_counts.withColumn("level", F.lit(3))
    
    topic4_counts = df.select(*group_cols, "daily_total", F.explode("topics4").alias("topic"))
    topic4_counts = topic4_counts.groupBy(group_cols).agg(F.sum("daily_total").alias(day))
    topic4_counts = topic4_counts.withColumn("level", F.lit(4))
    
    topic_counts = topic1_counts
    topic_counts = topic_counts.union(topic2_counts)
    topic_counts = topic_counts.union(topic3_counts)
    topic_counts = topic_counts.union(topic4_counts)
    
    topic_counts = topic_counts.filter((F.col("group").isNotNull()))
    # topic_counts.limit(10).show()
    
    # test for pizza
    # topic_counts.filter(F.lower("topic") == "pizza").limit(100).show()
    
    return topic_counts

In [5]:
# start with the first date
total_start = time.time()
daily_pageview_file, date = daily_pageview_files[0]
daily_pageview_combined = prepare(daily_pageview_file, date)

# iteratively join other days
for daily_pageview_file, date in daily_pageview_files[1:]:
    start = time.time()
    daily_pageview_combined = daily_pageview_combined.join(prepare(daily_pageview_file, date).repartition("group", "level"), on=group_cols + ["level"], how="outer")
    daily_pageview_combined = daily_pageview_combined.repartition("group", "level")
    print("processed %s (%d rows) in %.2f minutes" % (daily_pageview_file, daily_pageview_combined.count(), (time.time() - start) / (60)))

# 6M * 4 * langs X 365
    
# daily_pageview_combined.limit(10).show()
daily_pageview_combined.write.format("parquet").mode("overwrite").partitionBy("group", "level").save("../nvme/pageview_complete_per_topic_combined/2019.parquet")
print("done in %.2f hours" % ((time.time() - total_start) / (60**2)))

processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190102-user.parquet (4403582 rows) in 0.80 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190103-user.parquet (4558114 rows) in 0.91 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190104-user.parquet (4646457 rows) in 1.19 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190105-user.parquet (4714468 rows) in 1.48 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190106-user.parquet (4765553 rows) in 1.74 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190107-user.parquet (4805342 rows) in 2.03 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190108-user.parquet (4842547 rows) in 2.40 minutes
processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190109-user.parquet (4878634 rows) in 2.80 minutes
processed ../nvme/pagevi

21/10/27 13:03:38 WARN DAGScheduler: Broadcasting large task binary with size 1002.4 KiB


processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190118-user.parquet (5024666 rows) in 5.94 minutes


21/10/27 13:09:53 WARN DAGScheduler: Broadcasting large task binary with size 1005.9 KiB
21/10/27 13:09:57 WARN DAGScheduler: Broadcasting large task binary with size 1057.3 KiB


processed ../nvme/pageview_complete_processed/2019/2019-01/pageviews-20190119-user.parquet (5031106 rows) in 6.32 minutes


...

21/10/28 11:16:36 WARN DAGScheduler: Broadcasting large task binary with size 1005.9 KiB
21/10/28 11:16:45 WARN DAGScheduler: Broadcasting large task binary with size 1060.8 KiB
21/10/28 11:16:53 WARN DAGScheduler: Broadcasting large task binary with size 1115.8 KiB
21/10/28 11:17:02 WARN DAGScheduler: Broadcasting large task binary with size 1170.7 KiB
21/10/28 11:17:12 WARN DAGScheduler: Broadcasting large task binary with size 1225.6 KiB
21/10/28 11:17:21 WARN DAGScheduler: Broadcasting large task binary with size 1280.6 KiB
21/10/28 11:17:31 WARN DAGScheduler: Broadcasting large task binary with size 1335.5 KiB
21/10/28 11:17:39 WARN DAGScheduler: Broadcasting large task binary with size 1390.4 KiB
21/10/28 11:17:50 WARN DAGScheduler: Broadcasting large task binary with size 1445.3 KiB
21/10/28 11:17:58 WARN DAGScheduler: Broadcasting large task binary with size 1500.3 KiB
21/10/28 11:18:06 WARN DAGScheduler: Broadcasting large task binary with size 1555.2 KiB
21/10/28 11:18:14 WAR

[83787.762s][warning][gc,alloc] Executor task launch worker for task 15.0 in stage 1851597.0 (TID 1848743): Retried waiting for GCLocker too often allocating 252807 words
[83787.762s][warning][gc,alloc] spark-listener-group-appStatus: Retried waiting for GCLocker too often allocating 256 words


21/10/28 11:25:27 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
21/10/28 11:25:38 WARN DAGScheduler: Broadcasting large task binary with size 3.7 MiB
21/10/28 11:25:53 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
21/10/28 11:26:06 WARN DAGScheduler: Broadcasting large task binary with size 3.8 MiB
21/10/28 11:26:18 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
21/10/28 11:26:31 WARN DAGScheduler: Broadcasting large task binary with size 3.9 MiB
21/10/28 11:26:51 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
21/10/28 11:27:04 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB
21/10/28 11:27:19 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 11:27:32 WARN DAGScheduler: Broadcasting large task binary with size 4.1 MiB
21/10/28 11:27:53 WARN DAGScheduler: Broadcasting large task binary with size 4.2 MiB
21/10/28 11:28:08 WARN DAGScheduler: Broadcasting larg

processed ../nvme/pageview_complete_processed/2019/2019-03/pageviews-20190328-user.parquet (5137245 rows) in 42.50 minutes


KeyboardInterrupt: 

In [ ]:
sc.stop()